In [1]:
import pandas as pd
# import datawig

import pandas as pd
import numpy as np

from functions import calculate_OR, calculate_P



In [2]:

# Load dataset
data = pd.read_csv('./data/join_filled.csv')

x = data['ГГ'].copy()
y = data.iloc[:,114:].copy()

data = pd.concat([x, y], axis=1)
data = data.rename(columns={'ГГ': 'case'})
data['case'] = (data['case'] >= data['case'].mean()).astype(int)

In [3]:

# list of columns for which to calculate odds ratio
genotype_cols = [col for col in data.columns if col != 'case']

In [74]:

all_values = {}
# loop over genotype columns
for col in genotype_cols[:3]:
    # split into case and control groups
    case_group = data[data['case'] == 1][col]
    control_group = data[data['case'] == 0][col]

    # extract wild type and mutant type from column name

    w, m = col[-3:].split('>')
    wild, heterozygous, mutant = w+w, w+m, m+m

    # calculate odds ratio and confidence interval for dominant model
    dominant = calculate_OR(case_group, control_group, wild, heterozygous, mutant, model='dominant')
    recessive = calculate_OR(case_group, control_group, wild, heterozygous, mutant, model='recessive')
    codominant = calculate_OR(case_group, control_group, wild, heterozygous, mutant, model='codominant')
    allele = calculate_OR(case_group, control_group, wild, heterozygous, mutant, model='allele')

    p_all, p_gen = calculate_P(data['case'], data[col])

    all_values[col] = [
        *allele,
        *codominant,
        *dominant,
        *recessive
    ]

rs225131_NC_000001.11_8035426T>C   C   T
case                                    
1                                 25  57
0                                 23  55
0.42252639875536513 1.0
rs225132_NC_000001.11_8035440T>G   G   T
case                                    
1                                 25  57
0                                 23  55
0.42252639875536513 1.0
rs17367504_NC_000001.11_11802721A>G   A   G
case                                       
1                                    65  17
0                                    64  14
0.8506387241391525 0.8063733981650176


In [69]:
c

rs225131_NC_000001.11_8035426T>C,C,T
case,,
1,25,57
0,23,55


In [70]:
chi2_contingency(c)

Chi2ContingencyResult(statistic=0.0, pvalue=1.0, dof=1, expected_freq=array([[24.6, 57.4],
       [23.4, 54.6]]))

In [6]:
res = pd.DataFrame(all_values).T
res.columns = [
    'allele_M', 'allele_M_CI',
    'allele_W', 'allele_W_CI',
    'general_W', 'general_W_CI',
    'general_Ht', 'general_Ht_CI',
    'general_M', 'general_M_CI',
    'dominant_W', 'dominant_W_CI',
    'dominant_Ht_M', 'dominant_Ht_M_CI',
    'recessive_W_Ht', 'recessive_W_Ht_CI',
    'recessive_M', 'recessive_M_CI',
    ]
# res.reset_index(inplace=True)
# res.rename(columns={'index':'rsid'}, inplace=True)

In [7]:
res

,allele_M,allele_M_CI,allele_W,allele_W_CI,general_W,general_W_CI,general_Ht,general_Ht_CI,general_M,general_M_CI,dominant_W,dominant_W_CI,dominant_Ht_M,dominant_Ht_M_CI,recessive_W_Ht,recessive_W_Ht_CI,recessive_M,recessive_M_CI
rs225131_NC_000001.11_8035426T>C,1.0485,"(0.5045, 2.1853)",0.9537,"(0.4576, 1.982)",0.7462,"(0.2813, 1.9605)",1.669,"(0.6312, 4.4997)",0.4531,"(0.0387, 3.385)",0.7462,"(0.2813, 1.9605)",1.34,"(0.5101, 3.5549)",2.2069,"(0.2954, 25.8068)",0.4531,"(0.0387, 3.385)"


In [8]:
res.to_csv('./data/OR.csv')